In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from io import StringIO
import matplotlib.dates as dates
from datetime import datetime
import matplotlib.ticker as ticker
from random import shuffle
from time import time
from pandas.plotting import register_matplotlib_converters
%matplotlib notebook

#os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\CAP2020')
os.chdir('C:\\Users\\Daniel\\Python and R Notebooks\\CAP2020\\CAP2020')

## Import data

In [5]:
path = 'data\\processed\\compiled_quarterly_final_v2.csv'
df_quarter = pd.read_csv(path)
df_quarter.index = df_quarter['Quarter end']
df_quarter = df_quarter.drop('Quarter end', axis=1)
df_quarter.index.names = ['qend']


#df_bidlo.head()
#### TRANSPOSE DATA FOR EASE OF USE
# df_quarter = df_quarter.T
df_quarter.index = pd.to_datetime(df_quarter.index)
# print(df_quarter.shape)
df_quarter.head()

,Shares split adjusted,Price,EPS basic,EPS diluted,Dividend per share,Dividend payout ratio,P/E ratio,Long-term debt to equity ratio,Symbol,Quarter
qend,,,,,,,,,,
2019-09-30,67151778,63.99,0.59,0.58,0.035,0.0488,21.69,0.1859,AAN,2019Q3
2019-06-30,67547497,57.23,0.63,0.62,0.035,0.0464,19.94,0.1880,AAN,2019Q2
2019-03-31,67677449,48.63,0.83,0.82,0.035,0.0459,17.49,0.2251,AAN,2019Q1
2018-12-31,67202919,47.00,0.90,0.89,0.040,0.0455,10.83,0.1937,AAN,2018Q4
2018-09-30,68594784,48.86,0.64,0.62,0.030,0.0267,12.00,0.1686,AAN,2018Q3


## Drop duplicates

In [6]:
print('old shape {}'.format(df_quarter.shape))
df_quarter = df_quarter.drop_duplicates()
print('new shape: {}'.format(df_quarter.shape))

old shape (62252, 10)
new shape: (62252, 10)


## Change columns

In [7]:
df_quarter.columns = ['sharesout', 'price', 'eps_basic', 'eps_diluted', 'div_share', 'div_payout', 'p/e', 'debt/eq', 'symbol', 'quarter']
df_quarter.head(1)

,sharesout,price,eps_basic,eps_diluted,div_share,div_payout,p/e,debt/eq,symbol,quarter
qend,,,,,,,,,,
2019-09-30,67151778,63.99,0.59,0.58,0.035,0.0488,21.69,0.1859,AAN,2019Q3


## Change symbol to string

In [8]:
df_quarter['symbol'] = df_quarter['symbol'].astype(str)
df_quarter['quarter'] = df_quarter['quarter'].astype(str)

## Change numerical values to Int.\Float

In [9]:
df_quarter['sharesout'] = df_quarter['sharesout'].astype(int)
df_quarter['price'] = df_quarter['price'].astype(float)
df_quarter['eps_basic'] = df_quarter['eps_basic'].astype(float)
df_quarter['eps_diluted'] = df_quarter['eps_diluted'].astype(float)
df_quarter['div_share'] = df_quarter['div_share'].astype(float)
df_quarter['div_payout'] = df_quarter['div_payout'].astype(float)
df_quarter['p/e'] = df_quarter['p/e'].astype(float)
df_quarter['debt/eq'] = df_quarter['debt/eq'].astype(float)

## Link up the quarter to dates

In [10]:
df_quarter['new_key'] = df_quarter['symbol'] + '_'+ df_quarter['quarter']

In [14]:
dates_table = pd.read_csv('data\\external\\quarter_symbols.csv')
dates_table['std_date']  = pd.to_datetime(dates_table['std_date'])
dates_table['index'] = dates_table['Symbol'] + '_' + dates_table['Quarter']

In [15]:
dates_table.head()

,Quarter end,Symbol,Quarter,std_date,Unnamed: 4,Unnamed: 5,Unnamed: 6,2006Q4,12/31/2006,index
0,12/31/2018,AAN,2018Q4,2018-12-31,NaN,NaN,NaN,2007Q1,3/31/2007,AAN_2018Q4
1,9/30/2018,AAN,2018Q3,2018-09-30,NaN,NaN,NaN,2007Q2,6/30/2007,AAN_2018Q3
2,6/30/2018,AAN,2018Q2,2018-06-30,NaN,NaN,NaN,2007Q3,9/30/2007,AAN_2018Q2
3,3/31/2018,AAN,2018Q1,2018-03-31,NaN,NaN,NaN,2007Q4,12/31/2007,AAN_2018Q1
4,12/31/2017,AAN,2017Q4,2017-12-31,NaN,NaN,NaN,2008Q1,3/31/2008,AAN_2017Q4


In [16]:
df_merge = pd.merge(dates_table, df_quarter, left_on='index', right_on='new_key', how='inner')

In [17]:
df_merge.head()

,Quarter end,Symbol,Quarter,std_date,Unnamed: 4,Unnamed: 5,Unnamed: 6,2006Q4,12/31/2006,index,...,price,eps_basic,eps_diluted,div_share,div_payout,p/e,debt/eq,symbol,quarter,new_key
0,12/31/2018,AAN,2018Q4,2018-12-31,NaN,NaN,NaN,2007Q1,3/31/2007,AAN_2018Q4,...,47.00,0.90,0.89,0.04,0.0455,10.83,0.1937,AAN,2018Q4,AAN_2018Q4
1,9/30/2018,AAN,2018Q3,2018-09-30,NaN,NaN,NaN,2007Q2,6/30/2007,AAN_2018Q3,...,48.86,0.64,0.62,0.03,0.0267,12.00,0.1686,AAN,2018Q3,AAN_2018Q3
2,6/30/2018,AAN,2018Q2,2018-06-30,NaN,NaN,NaN,2007Q3,9/30/2007,AAN_2018Q2,...,43.87,0.55,0.54,0.03,0.0280,10.86,0.1564,AAN,2018Q2,AAN_2018Q2
3,3/31/2018,AAN,2018Q1,2018-03-31,NaN,NaN,NaN,2007Q4,12/31/2007,AAN_2018Q1,...,42.98,0.75,0.73,0.03,0.0278,10.61,0.2041,AAN,2018Q1,AAN_2018Q1
4,12/31/2017,AAN,2017Q4,2017-12-31,NaN,NaN,NaN,2008Q1,3/31/2008,AAN_2017Q4,...,39.67,2.51,2.45,0.03,0.0271,20.88,0.1572,AAN,2017Q4,AAN_2017Q4


## Filter Out Symbols that only traded for the entirety of 2007-2018

In [18]:
df_07_18 = df_quarter[df_quarter['quarter'].str.contains('2007') | df_quarter['quarter'].str.contains('2008')| df_quarter['quarter'].str.contains('2009')|
           df_quarter['quarter'].str.contains('2010') | df_quarter['quarter'].str.contains('2011')| df_quarter['quarter'].str.contains('2012')|
           df_quarter['quarter'].str.contains('2013') | df_quarter['quarter'].str.contains('2014')| df_quarter['quarter'].str.contains('2015')|
           df_quarter['quarter'].str.contains('2016') | df_quarter['quarter'].str.contains('2017')| df_quarter['quarter'].str.contains('2018')]

In [19]:
symbol_list = df_merge['symbol'].unique()

## Div Ratio

In [20]:
df_reduced = df_merge[['std_date', 'symbol', 'div_payout']]
########----------------#######
df_newq = pd.DataFrame([])

for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['div_payout']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)
    #df_temp.head()

df_newq.head()

std_date,2006-12-31,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,...,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31
AAN,NaN,0.0242,0.0250,0.0259,0.0269,0.0289,0.0279,0.0265,0.0249,0.0228,...,0.0543,0.0538,0.0556,0.0584,0.0271,0.0278,0.0280,0.0267,0.0455,NaN
AAPL,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.2643,0.2640,0.2627,0.2575,0.2505,0.2397,0.2326,0.2231,0.2276,NaN
AAP,NaN,0.1086,0.1054,0.1041,0.1016,0.0963,0.0921,0.0918,0.0959,0.0915,...,0.0385,0.0433,0.0477,0.0502,0.0373,0.0352,0.0332,0.0319,0.0414,NaN
AA,NaN,0.2527,0.2649,0.2690,0.2265,0.2592,0.2748,0.3152,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1813,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.6222,0.6030,0.5899,0.6014,0.7717,0.6588,0.7195,0.6693,1.0556,NaN


In [21]:
df_newq = df_newq.iloc[:, 1:-1]
df_newq.shape

(761, 48)

In [22]:
new_symbols_div = []
null_symbols_div = []
for symbol in df_newq.index:
    temp_value = df_newq.loc[symbol, :].isnull().sum()
    if temp_value == 0:
        new_symbols_div.append(symbol)
    else:
        null_symbols_div.append(symbol)

In [23]:
len(new_symbols_div), len(null_symbols_div)

(253, 508)

In [24]:
df_quarter_div = pd.DataFrame([])
for symbol in new_symbols_div:
    temp_df = df_newq[df_newq.index == symbol]
    df_quarter_div = pd.concat([df_quarter_div, temp_df])

df_quarter_div.shape

(253, 48)

In [26]:
export_path = 'data\\processed\\df_quarter_div_ratio07-18.csv'
df_quarter_div.to_csv(export_path)